In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests
import time
import json
#!pip install tweepy
import tweepy
#!pip install wptools
import wptools
from PIL import Image
from io import BytesIO

1. Directly download weratedogs twitter archive .csv file

In [2]:
tweets = pd.read_csv("../input/weratedogs-twitter-archive-enhanced/twitter-archive-enhanced.csv")
tweets.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


2. Download the tweet image prediction .tsv file using url

In [3]:
folder = 'tweet_tsv'
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

if not os.path.exists(folder):
    os.makedirs(folder)
    
r = requests.get(url)
with open(os.path.join(folder, url.split("/")[-1]), mode='wb') as file:
    response = file.write(r.content)

In [4]:
image = pd.read_csv('./tweet_tsv/image-predictions.tsv', sep='\t')
image.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


3. Use tweepy library to query additional data via the Twitter API
(tweet_json.txt):
    * Tweepy (twitter api) allows us to mine the data of any twitter user. Using .get_status of the API class in tweepy module fetch a status(tweet)

In [ ]:
consumer_key = 'mP9YUkyksJ3gGBcmLWWNjG1HK'
consumer_secret = 'O9joKKsymUtGXMg0rsUsr4uKm3J44y5nfadefJHm3sDWBget0b'
access_token = '1577688773621694465-rWwJYImL2LRjB37xJAHr51cxSECQ9G'
access_token_secret = '9Zvzb6dNMmAe8wTI9Ppzcsv7z63aRX5MgspDP9hHJOS0M' 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
# add `wait_on_rate_limit` parameter to automatically wait for rate limits to replenish
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True)

In [ ]:
start_time = time.time()

# Save all JSON data of each tweet and save the queries into a 'tweet_json.txt' file 
# so that you only need to run the following long queries once.
with open ('tweet_json.txt', 'w') as file:
    for tweet_id in tweets['tweet_id']:
        try:
            # Get status/tweet for each tweet_id
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            # Convert into .json string using .dumps() function 
            # Also, add '\n' as each tweet's JSON data to be written to its own line
            file.write(json.dumps(tweet) + "\n") 
        except Exception as e:
            # if error occurs, print tweet_id and error message
            print("No tweet found for {} with error message: {}".format(str(tweet_id), str(e)))
            
end_time = time.time()
print("The process finished in {} seconds".format(end_time - start_time))

In [5]:
# Read JSON line by line and create a python dictionary
info = []

with open('tweet_json.txt', 'r') as json_file:
    for line in json_file:
        # Parse JSON encoded/formatted string line by line
        json_data = json.loads(line)
        # Create a dictionary
        info.append({'tweet_id': json_data['id'], #call the value by stating 'key'
                    'favorites': json_data['favorite_count'],
                     'retweets': json_data['retweet_count'],
                     'timestamp': json_data['created_at']
                    })

# Create a dataframe
additional = pd.DataFrame(info, columns=['tweet_id', 'favorites', 'retweets', 'timestamp'])
additional.head(1)

,tweet_id,favorites,retweets,timestamp
0,892420643555336193,33592,6950,Tue Aug 01 16:23:56 +0000 2017


## Cleaning Data
In this section, clean all of the issues documented.

In [6]:
# Before any cleaning, make a copy of original datasets.
tweets_clean = tweets.copy()
image_clean = image.copy()
add_clean = additional.copy()

##### Clean 1 (all tables)
Define: 
- Drop retweet cases from all tables as this project will only analyse original ratings that have image (no retweets)

Code 1: merge tables

In [7]:
# Merge `text` column to image and additional table on='tweet_id'.
# To do so, first, create a separate table with identifier column `tweet_id` and `text`.
texts = tweets_clean[['tweet_id', 'text']]

# Merge
image_clean = pd.merge(image_clean, texts, on='tweet_id')
add_clean = pd.merge(add_clean, texts, on='tweet_id')

# Confirm
image_clean.columns, add_clean.columns

(Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
        'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'text'],
       dtype='object'),
 Index(['tweet_id', 'favorites', 'retweets', 'timestamp', 'text'], dtype='object'))

Code 2: remove retweets

In [8]:
# First, define a function that returns a mask of retweet entries.
def rt_mask(table):
    mask = table.text.str.contains("RT")
    return mask

# Drop retweet rows using the function.
tweets_clean = tweets_clean[~rt_mask(tweets_clean)]
image_clean = image_clean[~rt_mask(image_clean)]
add_clean = add_clean[~rt_mask(add_clean)]

In [17]:
# Confirm if there is any RT left in tables. You should get False's.
print(tweets_clean[tweets_clean['text'].str.contains('RT')].sum().any()),
print(image_clean[image_clean['text'].str.contains('RT')].sum().any()),
print(add_clean[add_clean['text'].str.contains('RT')].sum().any())

False
False
False


In [18]:
# Once it is confirmed, drop the redudant `text` columns from previously merged tables.
image_clean.drop('text', axis=1, inplace=True)
add_clean.drop('text', axis=1, inplace=True)

# Confirm. Column `text` should not be included in a duplicated series.
all_col = pd.Series(list(tweets_clean) + list(image_clean) + list(add_clean))
all_col[all_col.duplicated()]

17     tweet_id
29     tweet_id
32    timestamp
dtype: object

## 미완성이야아아
##### Clean () 
Define: 
- More rows than image (indicating there are tweets without pictures) 
- Other forms of Nan values(e.g. "None") 

Code 1:

In [27]:
# The `image_clean` table has 1986 rows while other tables have more than 2,000 rows.
# This might indicate there are tweets without image. 
# As we only want to investigate tweets with image, align the row numbers to image_clean table.

# Extract `tweet_id` from image_clean and make it into a list
id_list = list(image_clean.tweet_id)
# OR id_list = image_cleam.tweet_id.values.tolist()

# Create a function
def drop_rows(table, col):
    for a in table.col.values:
        if a not in id_list:
            table.drop(table.query('{0} == {1}'.format(col, a), axis=0, inplace=True))
        else:
            continue

In [29]:
drop_rows(tweets_clean, tweet_id)

NameError: name 'tweet_id' is not defined

##### Clean 2 (tweets_clean)
Define: 
- Join redundant columns into `stage` column with possible values of doggo, floofer, pupper, or puppo

Code:

> By running the code below, you can see that other than empty space or actual stage, "None" string value is included in the column value.

In [30]:
print("doggo: {}, floofer: {}, puppo: {}, pupper:{}"
     .format(tweets_clean.doggo.unique(), 
            tweets_clean.floofer.unique(),
            tweets_clean.puppo.unique(),
            tweets_clean.pupper.unique()))

doggo: ['None' 'doggo'], floofer: ['None' 'floofer'], puppo: ['None' 'puppo'], pupper:['None' 'pupper']


In [35]:
# So before joining the four columns, first replace "None" to empty-space "".
# Then join the columns.
# Some columns for example will be "   floofer ", reflecting empty spaces created by joining the columns. So strip those empty spaces.
tweets_clean['stage'] = (tweets_clean[['doggo', 'floofer', 'puppo', 'pupper']]
                         .replace("None","").apply(lambda x: " ".join(x), axis=1).str.strip())

# Confirm if merge successful.
tweets_clean.query("stage != ''").head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,stage
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None,doggo


In [36]:
# Drop the redundant columns
tweets_clean.drop(columns = ['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

# Confirm
tweets_clean.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
stage                          object
dtype: object

##### Clean 3 (all tables)
Define: 
- Convert wrong datatypes

Code:

In [37]:
# Create a function that converts wrong datatypes with input arguments (table)
def change_dtypes(table):
    for col in table.columns:
        if col[-3:] == "_id":
            table[col] = table[col].astype(str)
        elif col[-5:] == "stamp":
            table[col] = pd.to_datetime(table[col])
        else:
            table[col] = table[col]
            
# Call the function
change_dtypes(tweets_clean)
change_dtypes(image_clean)
change_dtypes(add_clean)

In [38]:
# Confirm
tweets_clean.dtypes, image_clean.dtypes, add_clean.dtypes

(tweet_id                                   object
 in_reply_to_status_id                      object
 in_reply_to_user_id                        object
 timestamp                     datetime64[ns, UTC]
 source                                     object
 text                                       object
 retweeted_status_id                        object
 retweeted_status_user_id                   object
 retweeted_status_timestamp         datetime64[ns]
 expanded_urls                              object
 rating_numerator                            int64
 rating_denominator                          int64
 name                                       object
 stage                                      object
 dtype: object,
 tweet_id     object
 jpg_url      object
 img_num       int64
 p1           object
 p1_conf     float64
 p1_dog         bool
 p2           object
 p2_conf     float64
 p2_dog         bool
 p3           object
 p3_conf     float64
 p3_dog         bool
 dtype: object,
 t

##### Clean 4 (tweets_clean)
Define: 
- Split the table into two separate tables since the tweets_clean table contains two different units of information

Code:

In [40]:
tweets_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'stage'],
      dtype='object')

In [41]:
# Create a new dataframe
ratings_col = ['tweet_id', 'rating_numerator', 'rating_denominator', 'name', 'stage']
ratings = tweets_clean[ratings_col]

# Drop redundant columns except for `tweet_id`
tweets_clean.drop(columns = ratings_col[1:], inplace=True)

In [42]:
# Confirm
tweets_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls'],
      dtype='object')

##### Clean 5 (image_clean)
Define: 
- Create an new column for actual breed of a dog, not falsely predicted breed.

Code:
- np.where(condition, value if condition True, value if condition False)
- Since the prediction p1, p2, p3 is hierarchical, if the first prediction is True (i.e. is a dog), that is the prediction you want. If False (i.e. not a dog), move onto the second prediction
- `px_dog` is vectorised, so you can just use the column itself as a condition in np.where() 

In [48]:
image_clean['dog_breed'] = np.where(image_clean['p1_dog'], image_clean['p1'],
                                   np.where(image_clean['p2_dog'], image_clean['p2'],
                                           np.where(image_clean['p3_dog'], image_clean['p3'],
                                           np.nan)))

In [49]:
# Confirm
image_clean.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,dog_breed
738,687124485711986689,https://pbs.twimg.com/media/CYkoE10WEAAWqxm.jpg,1,car_mirror,0.997121,False,seat_belt,0.000375,False,beagle,0.000216,True,beagle
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False,NaN
1053,714214115368108032,https://pbs.twimg.com/media/Cell8ikWIAACCJ-.jpg,1,pug,0.533967,True,bloodhound,0.164826,True,German_shepherd,0.046524,True,pug


##### Clean 6 (all tables)
Define: 
- Drop any duplicate columns amongst tables

Code:

In [54]:
# Look for duplicated columns
all_cols = pd.Series(list(tweets_clean) + list(image_clean) + list(add_clean))
all_cols[all_cols.duplicated()]

10     tweet_id
23     tweet_id
26    timestamp
dtype: object

In [56]:
# Drop `timestamp` column from `add_clean` table as it is irrelevant to table's observational unit
add_clean.drop('timestamp', axis=1, inplace=True)

# Confirm
add_clean.columns

Index(['tweet_id', 'favorites', 'retweets'], dtype='object')

##### Clean (image_clean)
Define: 
- Lowercase all breed values in `p1`, `p2`, `p3` columns
- Lowercase `dog_breed` column 
- Standardise the length of decimal numbers in `p1_conf`, `p2_conf`, `p3_conf`: .round()

Code 1:

In [57]:
tweets_clean.to_csv('tweets_clean.csv', index=False)
image_clean.to_csv('image_clean.csv', index=False)
add_clean.to_csv('add_clean.csv', index=False)